# Transformation from Equatorial Coordinates to Horizontal Coordinates
by Sergio Nuñez

I started uploading our cvs data to python using Pandas framework. This framework allows me to have a better manipulation of it.

In [1]:
import pandas as pd
data = pd.read_csv ('/Users/usuario/Python/Astronomy/Supernovae/tns_search.csv')
data[:3]

,ID,Name,RA,DEC,Obj. Type,Redshift,Host Name,Host Redshift,Reporting Group/s,Discovery Data Source/s,...,Class. Instrument/s,TNS AT,Public,End Prop. Period,Discovery Mag/Flux,Discovery Filter,Discovery Date (UT),Sender,Remarks,Ext. catalog/s
0,65308,SN 2020rod,21:47:50.615,-11:52:04.61,SN Ia,0.090,NaN,NaN,ZTF,ZTF,...,P60 - SEDM,1,1,NaN,19.93,g-ZTF,2020-08-08 08:02:24.000,ZTF_Bot1,NaN,NaN
1,65286,SN 2020rni,23:57:46.652,+03:57:45.20,SN Ia,0.094,SDSS J235746+035747,0.094,SIRAH,ZTF,...,SALT - RSS,1,1,NaN,20.72,g-ZTF,2020-08-14 09:48:08.000,Jha,NaN,NaN
2,65251,SN 2020rlz,00:28:28.304,+48:51:31.57,SN Ia,0.060,NaN,NaN,ZTF,ZTF,...,P60 - SEDM,1,1,NaN,19.51,r-ZTF,2020-08-04 08:48:28.800,ZTF_Bot1,NaN,NaN


I defined some useful functions:
    1. hms2dec: Transform HMS notation to degrees (HMS notations is very common in right ascention).
    2. dms2dec: Transform DMS notation to degrees (DMS notation is very common in declination).
    3. RA_to_deg: This loop allows to transform all the row of RA to degrees.
    4. DEC_to_deg: This loop allow to transform all the row of DEC to degrees.

In [2]:
#Useful functions
def hms2dec(h,m,s):
    r_a = 15*(h + m/60 + s/3600)
    return r_a

def dms2dec(d,m,s):
    dec = abs(d) + m/60 + s/3600
    sign = d/abs(d)
    return sign*dec

#Right Ascention in degrees
def RA_to_deg(data):
    ra_str_in_deg = []

    for ra_raw in data:
        #print(ra_raw)
        tempo1 = ra_raw.split(':')
        #print(tempo1)
        h=float(tempo1[0])
        m=float(tempo1[1])
        s=float(tempo1[2])
        
        ra_deg = hms2dec(h,m,s)
    
        ra_str_in_deg.append(ra_deg)
    
    return ra_str_in_deg

#Declination in degrees
def DEC_to_deg(data):
    dec_str_in_deg = []
    
    for dec_raw in data:
        tempo1 = dec_raw.split(':')
        #print(tempo1)
        d = float(tempo1[0])
        m = float(tempo1[1])
        s = float(tempo1[2])
    
        dec_deg = dms2dec(d,m,s)
    
        dec_str_in_deg.append(dec_deg)

    return dec_str_in_deg

I take only what I want, right ascention (RA) and declination (DEC), which will store in ra and dec, respectivetly. And then, I use the above functions and Pandas data frame to find right ascention and declination in degrees. They are store in ra_in_deg and dec_in_deg, respectivetly.

In [3]:
ra=data['RA']
dec=data['DEC']

In [25]:
ra_in_deg = pd.DataFrame(RA_to_deg(ra), columns=['RA'])
ra_in_deg[:3]

,RA
0,326.960896
1,359.444383
2,7.117933


In [12]:
dec_in_deg = pd.DataFrame(DEC_to_deg(dec), columns=['DEC'])
dec_in_deg[:3]

,DEC
0,-11.867947
1,3.962556
2,48.858769


Short example of how I'll work with the rows of ra_in_deg and dec_in_deg

In [34]:
ra_in_deg.loc[:]['RA'] #they're float

0      326.960896
1      359.444383
2        7.117933
3      345.290037
4      358.749854
          ...    
121    198.722667
122    251.645250
123    185.015708
124    239.679542
125    338.350029
Name: RA, Length: 126, dtype: float64

The fun starts now. I import astropy commands for the transformation from equatorial coordinates to horizontal coordinates.

In [30]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import EarthLocation
from astropy.time import Time
import numpy as np

With SkyCoord I upload all the data of right ascention and declination to be worked in astorpy. They are stored in test.
With EarthLocation I define the position of the observatory. It is stored in moquegua.

In [35]:
test = SkyCoord(ra_in_deg.loc[:]['RA']*u.deg, dec_in_deg.loc[:]['DEC']*u.deg, frame='icrs')
#print(test)

In [39]:
moquegua = EarthLocation(lat=-16.828161*u.deg, lon=-70.678569*u.deg, height=3300*u.m)

In [40]:
observing_time = ('2020-08-30 23:00') #18:00 for Peru

In [41]:
from astropy.coordinates import AltAz

aa = AltAz(location=moquegua, obstime=observing_time)
print(aa)

<AltAz Frame (obstime=2020-08-30 23:00:00.000, location=(2021560.10303719, -5765754.96312368, -1835594.27902581) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron)>


In [44]:
alt = test.transform_to(aa).alt
az = test.transform_to(aa).az

In [48]:
az[0]

<Longitude 96.92888434 deg>